In [72]:
import tensorflow as tf
print(tf.__version__)

2.15.0


# Problem Statement

# Data Preprocessing

In [73]:
import tensorflow as tf
from tensorflow import keras

train = tf.keras.utils.image_dataset_from_directory('CNN Data/train', labels = 'inferred')
val = tf.keras.utils.image_dataset_from_directory('CNN Data/val', labels = 'inferred')

Found 9999 files belonging to 10 classes.
Found 2000 files belonging to 10 classes.


# Deep Learning Model: CNN

In [74]:
types = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [75]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),

    tf.keras.layers.Conv2D(16, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),


    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10)#, activation=tf.nn.softmax)
])

In [76]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy') > 0.75):
      print("Cancelling training.")
      self.model.stop_training = True

callbacks = myCallback()

In [77]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [78]:
model.fit(train, validation_data=val, epochs=60, callbacks=[callbacks])

Epoch 1/60


313/313 [==============================] - 64s 198ms/step - loss: 2.2347 - accuracy: 0.1691 - val_loss: 2.1931 - val_accuracy: 0.2000
Epoch 2/60
313/313 [==============================] - 62s 197ms/step - loss: 2.1345 - accuracy: 0.2164 - val_loss: 2.0767 - val_accuracy: 0.2595
Epoch 3/60
313/313 [==============================] - 62s 198ms/step - loss: 2.0715 - accuracy: 0.2458 - val_loss: 2.0255 - val_accuracy: 0.2710
Epoch 4/60
313/313 [==============================] - 62s 197ms/step - loss: 2.0218 - accuracy: 0.2761 - val_loss: 1.9951 - val_accuracy: 0.3010
Epoch 5/60
313/313 [==============================] - 62s 197ms/step - loss: 1.9819 - accuracy: 0.2896 - val_loss: 2.0038 - val_accuracy: 0.2920
Epoch 6/60
313/313 [==============================] - 62s 197ms/step - loss: 1.9518 - accuracy: 0.3044 - val_loss: 1.9807 - val_accuracy: 0.3070
Epoch 7/60
313/313 [==============================] - 62s 197ms/step - loss: 1.9360 - accuracy: 0.3058 - val_loss: 1.8969 - val_accuracy: 0.3

KeyboardInterrupt: 

# Evaluation of Model

In [ ]:
model.evaluate(val)

63/63 [==============================] - 3s 49ms/step - loss: 2.1981 - accuracy: 0.2265


[2.198082447052002, 0.226500004529953]

# Hyperparameter Tuning